<a href="https://colab.research.google.com/github/musombaangel/feedback-analysis/blob/main/Course_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
#relevant imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from datasets import Dataset
import torch
import pandas as pd
import numpy as np
import requests
import re

In [ ]:
df=pd.read_csv('/content/Extended_Course_Feedback_Dataset.csv')
df.head()

,COMMENT,LABEL
0,"""This course exceeded my expectations in every...",very positive
1,"""The instructor was incredibly knowledgeable a...",very positive
2,"""I learned so much and feel very confident in ...",very positive
3,"""The course materials were comprehensive and e...",very positive
4,"""The interactive sessions made learning fun an...",very positive


In [ ]:
df['LABEL'].replace('very positive',5, inplace=True)
df['LABEL'].replace('positive',4, inplace=True)
df['LABEL'].replace('neutral',3, inplace=True)
df['LABEL'].replace('negative',2, inplace=True)
df['LABEL'].replace('very negative',1, inplace=True)

In [ ]:
#Changing the labels of the columns
df = df.rename(columns={'COMMENT': 'text', 'LABEL': 'labels'})

In [ ]:
df.head()

,text,labels
0,"""This course exceeded my expectations in every...",5
1,"""The instructor was incredibly knowledgeable a...",5
2,"""I learned so much and feel very confident in ...",5
3,"""The course materials were comprehensive and e...",5
4,"""The interactive sessions made learning fun an...",5


In [ ]:
x_train,x_test,y_train,y_test=train_test_split(df['text'],df['labels'],test_size=0.25,random_state=42)

In [ ]:
#putting the training and test data together
training_data=list(zip(x_train,y_train))
test_data=list(zip(x_test,y_test))

In [ ]:
training_data = pd.DataFrame(training_data,columns=['text','labels'])
test_data=pd.DataFrame(test_data,columns=['text','labels'])

In [ ]:
training_data = Dataset.from_pandas(training_data)
test_data = Dataset.from_pandas(test_data)

AttributeError: 'Dataset' object has no attribute 'columns'

In [ ]:
print(training_data.shape)
print(test_data.shape)

(1500, 2)
(500, 2)


In [ ]:
print('Train data types \n',training_data.features)
print('Test data types \n',test_data.features)

Train data types 
 {'text': Value(dtype='string', id=None), 'labels': Value(dtype='int64', id=None)}
Test data types 
 {'text': Value(dtype='string', id=None), 'labels': Value(dtype='int64', id=None)}


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

In [ ]:
values_train=training_data['labels']
encoded_data_train=[]
for data in values_train:
  data=str(data)
  encoded=tokenizer(data,padding=True,truncation=True, max_length=120)
  encoded_data_train.append(encoded)
print(encoded_data_train[0])


{'input_ids': [101, 123, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


In [ ]:
values_test=test_data['labels']
encoded_data_test=[]
for data in values_test:
  data=str(data)
  encoded=tokenizer(data,padding=True,truncation=True, max_length=120)
  encoded_data_test.append(encoded)
print(encoded_data_test[0])

{'input_ids': [101, 123, 102], 'token_type_ids': [0, 0, 0], 'attention_mask': [1, 1, 1]}


In [ ]:
training_data['text']=encoded_data_train
test_data['text']=encoded_data_test


TypeError: 'Dataset' object does not support item assignment

In [ ]:
df[['input_ids', 'token_type_ids', 'attention_mask']] = pd.DataFrame(df['text'].tolist(), index=df.index)
df.head()

,text,labels,input_ids,token_type_ids,attention_mask
0,"[input_ids, token_type_ids, attention_mask]",5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"
1,"[input_ids, token_type_ids, attention_mask]",5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"
2,"[input_ids, token_type_ids, attention_mask]",5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"
3,"[input_ids, token_type_ids, attention_mask]",5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"
4,"[input_ids, token_type_ids, attention_mask]",5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"


In [ ]:
df.drop('text',axis=1,inplace=True)
df.head()

,labels,input_ids,token_type_ids,attention_mask
0,5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"
1,5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"
2,5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"
3,5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"
4,5,"[1, 1, 1]","[101, 126, 102]","[0, 0, 0]"


In [ ]:
df.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

NameError: name 'df' is not defined

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-multilingual-uncased", num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="epoch",
    remove_unused_columns=False
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
print(type(training_data))

<class 'datasets.arrow_dataset.Dataset'>


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= training_data,
    eval_dataset=test_data
)

In [ ]:
trainer.train()

ValueError: You have to specify either input_ids or inputs_embeds

In [ ]:
#ignore this one for now
from torch.utils.data import Dataset

class PandasDataset(Dataset):
    def __init__(self, dataframe, tokenizer, text_column='text', label_column='label', max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.text_column = text_column
        self.label_column = label_column
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = row[self.text_column]
        label = row[self.label_column]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),  # Remove batch dimension
            'attention_mask': encoding['attention_mask'].squeeze(),  # Remove batch dimension
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = PandasDataset(training_data, tokenizer)
eval_dataset = PandasDataset(test_data, tokenizer)